In [ ]:
# Install dependencies
!pip install transformers datasets langchain
!pip install langchain langchain-community


# Load model & tokenizer
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base")
llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:

# Sample prompt setup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """
You are a clinical assistant AI. Answer the medical question accurately.

Question: {question}
Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
chain = LLMChain(prompt=prompt, llm=llm)

# Run a test example
question = "What is the treatment for mild asthma?"
print(chain.run({"question": question}))


In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

def llm_model(prompt_txt, model_name="google/flan-t5-base", params=None):
    # Default generation parameters
    default_params = {
        "max_new_tokens": 256,
        "min_new_tokens": 0,  # Not used directly
        "temperature": 0.5,
        "top_p": 0.9,
        "top_k": 50
    }

    if params:
        default_params.update(params)

    # Hugging Face text2text-generation pipeline
    pipe = pipeline(
        task="text2text-generation",
        model=model_name,
        max_new_tokens=default_params["max_new_tokens"],
        temperature=default_params["temperature"],
        top_p=default_params["top_p"],
        top_k=default_params["top_k"],
        do_sample=True
    )

    # Wrap in LangChain's HuggingFace wrapper
    llm = HuggingFacePipeline(pipeline=pipe)

    # Return response using LangChain's interface
    response = llm.invoke(prompt_txt)
    return response


let's introduce a basic prompt that utilizes specific parameters to guide the language model's response. You'll then define a simple prompt and retrieve the model's response,

The prompt used is "The wind is". Let the model generate itself.


In [ ]:
prompt = """
You are a clinical assistant AI.
Question: What is the first-line treatment for mild hypertension?
Answer:
"""

print(llm_model(prompt))


In [ ]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 10,  # Not used directly by HF, but kept for compatibility
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1
}

# ➤ Prompt example (you can change this)
prompt = "What is the recommended antibiotic for a urinary tract infection in a non-pregnant woman?"

# ➤ Generate response
response = llm_model(prompt, params = params)

# ➤ Output
print(f"Prompt:\n{prompt}\n")
print(f"Response:\n{response}\n")

You might notice that the response may appear truncated or incomplete. This is because you have set the `max_new_tokens,` which restricts the number of tokens the model can generate.

Try to adjust the parameters and observe the difference in the response.


### Zero-shot prompt

Here is an example of a zero-shot prompt.

Zero-shot learning is crucial for testing a model's ability to apply its pre-trained knowledge to new, unseen tasks without additional training. This capability is valuable for gauging the model's generalization skills.

In this example, let's demonstrate a zero-shot learning scenario using a prompt that asks the model to classify a statement without any prior specific training on similar tasks. The prompt requests the model to assess the truthfulness of the statement: "The Eiffel Tower is located in Berlin.". After defining the prompt, you'll execute it with default parameters and print the response.

This approach helps you understand how well the model can handle direct questions based on its underlying knowledge and reasoning abilities.


In [ ]:
prompt = """Classify the following statement as true or false:
            'The Eiffel Tower is located in Berlin.'

            Answer:
"""
response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

### One-shot prompt

Here is a one-shot learning example where the model is given a single example to help guide its translation from English to French.

The prompt provides a sample translation pairing, "How is the weather today?" translated to "Comment est le temps aujourd'hui?" This example serves as a guide for the model to understand the task context and desired format. The model is then tasked with translating a new sentence, "Where is the nearest supermarket?" without further guidance.


In [ ]:
params = {
    "max_new_tokens": 20,
    "temperature": 0.1,
}

prompt = """Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”

            Now, translate the following sentence from English to French:

            English: “Where is the nearest supermarket?”

"""
response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

The model's response shows how it applies the structure and context provided by the initial example to translate the new sentence.

Consider experimenting with different sentences or adjusting the parameters to see how these changes impact the model's translations.


### Few-shot prompt

Here is an example of few-shot learning by classifying emotions from text statements.

Let's provide the model with three examples, each labeled with an appropriate emotion—joy, frustration, and sadness—to establish a pattern or guideline on how to categorize emotions in statements.

After presenting these examples, let's challenge the model with a new statement: "That movie was so scary I had to cover my eyes." The task for the model is to classify the emotion expressed in this new statement based on the learning from the provided examples.


In [ ]:
 #parameters  `max_new_tokens` to 10, which constrains the model to generate brief responses

params = {
    "max_new_tokens": 10,
}

prompt = """Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy

            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration

            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness

            Now, classify the emotion in the following statement:
            Statement: 'That movie was so scary I had to cover my eyes.’


"""
response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

The parameters are set with `max_new_tokens` to 10, which constrains the model to generate brief responses, focusing on the essential output without elaboration.

The model's response demonstrates its ability to use the provided few examples to understand and classify the emotion of the new statement effectively following the same pattern in examples.


### Chain-of-thought (CoT) prompting

Here is an example of the Chain-of-Thought (CoT) prompting technique, designed to guide the model through a sequence of reasoning steps to solve a problem. In this example, the problem is a simple arithmetic question: “A store had 22 apples. They sold 15 apples today and received a new delivery of 8 apples. How many apples are there now?”

The CoT technique involves structuring the prompt by instructing the model to “Break down each step of your calculation.” This encourages the model to include explicit reasoning steps, mimicking human-like problem-solving processes.


In [ ]:
params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples.
            How many apples are there now?’

            Break down each step of your calculation

"""
response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

In [ ]:
params = {
    "max_new_tokens": 128,
    "temperature": 1.0,   # Neutral
    "top_p": 1.0,
    "top_k": 50
}

prompt = """You are a helpful assistant.

Solve this problem step-by-step:

"A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. How many apples are there now?"

Step-by-step solution:"""

response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

### Self-consistency

This example demonstrates the self-consistency technique in reasoning through multiple calculations for a single problem. The problem posed is: “When I was 6, my sister was half my age. Now I am 70, what age is my sister?”

The prompt instructs, “Provide three independent calculations and explanations, then determine the most consistent result.” This encourages the model to engage in critical thinking and consistency checking, which are vital for complex decision-making processes.


In [ ]:
params = {
    "max_new_tokens": 512,
}

prompt = """When I was 6, my sister was half of my age. Now I am 70, what age is my sister?
See this as an example:When you were 10, your sister was 5 → age gap = 5 years Now you are 70 → sister = 70 - 5 = 65 years old.

            Now Provide three independent calculations and explanations, then determine the most consistent result.

"""
response = llm_model(prompt, params = params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

This model doesnt show good result. We will use google/flan-t5-xl

In [ ]:
params = {
    "max_new_tokens": 512,
}

prompt = """When I was 6, my sister was half of my age. Now I am 70, what age is my sister?
See this as an example:When you were 10, your sister was 5 → age gap = 5 years Now you are 70 → sister = 70 - 5 = 65 years old.

            Now Provide three independent calculations and explanations, then determine the most consistent result.

"""

response = llm_model(prompt, model_name="google/flan-t5-xl", params=params)

print(f"prompt:\n{prompt}\n")
print(f"response:\n{response}\n")


The model's response shows that it provides three different calculations and explanations. Each calculation attempts to derive the sister's age using different logical approaches.

Self-consistency can help identify the most accurate and reliable answer in scenarios where multiple plausible solutions exist.


### Prompt template

[Prompt template](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) is a key concept in langchain, it helps to translate user input and parameters into instructions for a language model. This can be used to guide a model's response, helping it understand the context and generate relevant and coherent language-based output.

Use the `PromptTemplate` to create a template for a string-based prompt. In this template, you'll define two parameters: `adjective` and `content`. These parameters allow for the reuse of the prompt across different situations. For instance, to adapt the prompt to various contexts, simply pass the relevant values to these parameters.


In [ ]:
from langchain.prompts import PromptTemplate
template = "Tell me a {adjective} joke about {content}."
prompt = PromptTemplate.from_template(template)
formatted_prompt = prompt.format(adjective="funny", content="chickens")
print(formatted_prompt)


In [ ]:
formatted_prompt = prompt.format(adjective="funny", content="chickens")
print(formatted_prompt)


In [ ]:
response = llm_model(formatted_prompt, model_name="google/flan-t5-base")
print(response)


In [ ]:
# Create and use a PromptTemplate
template = "Tell me a {adjective} joke about {content}."
prompt = PromptTemplate.from_template(template)
formatted_prompt = prompt.format(adjective="sarcastic", content="doctors")

# Call the model
response = llm_model(formatted_prompt)
print("Prompt:", formatted_prompt)
print("Response:", response)

In [ ]:
# 2. Imports
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
# 3. Define the model wrapper using HuggingFace
def llm_model(model_name="google/flan-t5-base", params=None):
    default_params = {
        "max_new_tokens": 128,
        "temperature": 0.7,
        "top_p": 0.9,
        "top_k": 50
    }
    if params:
        default_params.update(params)

    task = "text2text-generation" if "t5" in model_name.lower() else "text-generation"

    pipe = pipeline(
        task=task,
        model=model_name,
        tokenizer=model_name,
        max_new_tokens=default_params["max_new_tokens"],
        temperature=default_params["temperature"],
        top_p=default_params["top_p"],
        top_k=default_params["top_k"],
        do_sample=True
    )

    return HuggingFacePipeline(pipeline=pipe)


In [ ]:
# 4. Define the prompt template
template = "Tell me a {adjective} joke about {content}."
prompt = PromptTemplate.from_template(template)
# 5. Create the LLMChain
llm = llm_model()  # flan-t5-base by default
llm_chain = LLMChain(prompt=prompt, llm=llm)
# 6. Run the chain
response = llm_chain.invoke({"adjective": "funny", "content": "chickens"})
print("Generated Joke:\n", response["text"])

In [ ]:
response = llm_chain.invoke(input = {"adjective": "sad", "content": "fish"})
print(response["text"])

# 🤖 What is LLMChain in LangChain?

## 🔍 Manual Prompting (The Basic Way)

Manual prompting means you prepare your prompt manually as a string, insert values yourself, send it to the model, and get back a plain response.

### Example:
You write:
> “Tell me a funny joke about chickens.”

You send this directly to the model, and it returns a joke.

### 🔹 Characteristics:
- You manually insert values into the prompt (e.g., using `.format()` or f-strings)
- You call the model directly with a plain string
- It's simple and works fine for one-off tests
- ❌ Not scalable
- ❌ Not reusable or modular

---

## ✅ What is `LLMChain`?

`LLMChain` is a LangChain abstraction that combines:
1. A **PromptTemplate** (e.g., `"Tell me a {adjective} joke about {content}"`)
2. A **language model** (like Hugging Face, OpenAI, or Anthropic)

LangChain automatically:
- Fills in variables (e.g., `{"adjective": "funny", "content": "chickens"}`)
- Generates the final prompt
- Sends it to the model
- Returns a **structured response**

---

## 🧩 Why Use `LLMChain`?

### Advantages:
- ✅ **Reusable** — Define once, use many times with different inputs
- ✅ **Structured** — Pass inputs as dictionaries, get outputs with keys like `"text"`
- ✅ **Composable** — Can be combined with memory, RAG, agents, etc.
- ✅ **Clean Code** — No manual prompt formatting or string building

---

## 🧠 When to Use `LLMChain`

| Use Case                        | Use `LLMChain`? |
|---------------------------------|-----------------|
| Quick one-off prompt            | ❌ Optional     |
| Prompt testing/experimentation  | ✅ Yes          |
| Chatbots or assistants          | ✅ Absolutely   |
| Retrieval-augmented generation  | ✅ Yes          |
| Tool-using agents               | ✅ Required     |
| Building AI pipelines or apps   | ✅ Always       |

---

## 🔚 Summary

- Manual prompting = simple but limited
- `LLMChain` = structured, reusable, and essential for real apps
- Use `LLMChain` when building anything beyond toy examples

It’s the standard way to plug LLMs into real-world workflows in LangChain.



### Text summarization

Here is a text summarization agent designed to help summarize the content you provide to the LLM.

You can store the content to be summarized in a variable, allowing for repeated use of the prompt.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Your content
content = """
The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation.
Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems.
For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion.
Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers.
These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

# Use a better-structured prompt
template = """You are a helpful assistant.

Summarize the following content in one sentence:

{content}
"""

prompt = PromptTemplate.from_template(template)

# Assuming you are using flan-t5-base or mistral llm
llm_chain = LLMChain(prompt=prompt, llm=llm_model(model_name="google/flan-t5-base"))

# Invoke with structured input
response = llm_chain.invoke({"content": content})

# Print the result
print("Summary:\n", response["text"])


### Question answering

Here is a Q&A agent.

This agent enables the LLM to learn from the provided content and answer questions based on what it has learned. Occasionally, if the LLM does not have sufficient information, it might generate a speculative answer. To manage this, you'll specifically instruct it to respond with "Unsure about the answer" if it is uncertain about the correct response.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define content and question
content = """
The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets.
The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.
The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

# Properly structured prompt
template = """
You are a helpful assistant.

Question: {question}

Content:
{content}

If you're unsure about the answer, reply: "Unsure about answer".

Answer:
"""

# Create prompt and chain
prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm_model(model_name="google/flan-t5-base"), output_key="answer")

# Run the chain
response = llm_chain.invoke({"question": question, "content": content})

# Output the result
print("Answer:\n", response["answer"])


### Text classification

Here is a text classification agent designed to categorize text into predefined categories. This example employs zero-shot learning, where the agent classifies text without prior exposure to related examples.

Can you revise it to the one-shot learning or few-shot learning in the exercises?


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

text = """
The concert last night was an exhilarating experience with outstanding performances by all artists.
"""

categories = "Entertainment, Food and Dining, Technology, Literature, Music."

# Improved prompt structure
template = """
You are a text classification assistant.

Classify the following text into one of the given categories.

Text:
{text}

Categories:
{categories}

Category:
"""

# Create prompt template
prompt = PromptTemplate.from_template(template)

# Create LLMChain with flan-t5-base
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm_model(model_name="google/flan-t5-base"),
    output_key="category"
)

# Run the chain
response = llm_chain.invoke({"text": text, "categories": categories})

# Output the result
print("Predicted Category:", response["category"])


### Code generation

Here is an example of an SQL code generation agent. This agent is designed to generate SQL queries based on given descriptions. It interprets the requirements from your input and translates them into executable SQL code.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Natural language description
description = """
Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days.
The table 'purchases' contains a column 'purchase_date'.
"""

# Clear, structured prompt template
template = """
You are an expert SQL generator.

Task:
Generate an SQL query based on the following description:

{description}

Only output the SQL query.

SQL Query:
"""

# Create prompt and chain
prompt = PromptTemplate.from_template(template)
llm_chain = LLMChain(prompt=prompt, llm=llm_model(model_name="google/flan-t5-base"), output_key="query")

# Run the chain
response = llm_chain.invoke({"description": description})

# Print output
print("Generated SQL Query:\n", response["query"])


### Role playing

You can also configure the LLM to assume specific roles as defined by us, enabling it to follow predetermined rules and behave like a task-oriented chatbot.

For example, the code below configures the LLM to act as a game master. In this role, the LLM answers questions about games while maintaining an engaging and immersive tone, enhancing the user experience.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Inputs
role = "game master"
tone = "engaging and immersive"
question = "Can you describe the scene as the players enter the haunted castle?"

# Clear prompt template
template = """
You are an expert {role}.

Task:
I have the following question: "{question}"

Your response should be written in a {tone} tone.

Answer:
"""

# Create prompt
prompt = PromptTemplate.from_template(template)

# Initialize LLMChain with mixtral_llm (replace with your llm_model() if needed)
llm_chain = LLMChain(prompt=prompt, llm=llm_model(model_name="google/flan-t5-base"), output_key="answer")

# Run the chain
response = llm_chain.invoke({
    "role": role,
    "tone": tone,
    "question": question
})

# Print result
print("Response:\n", response["answer"])
